In [52]:
from ultralytics import YOLO
model = YOLO("modelFolder/model_v7.pt")
results = model(source="validimage/5_Color.png",device="cpu", project = "results",name="test6d", conf=0.8 )
for result in results:
    image_points = result.masks.xyn
    print(len(image_points))
    # xy = result.masks.xy
    # print(image_points)
    # print(xy)
    


image 1/1 c:\Team_Project-1\validimage\5_Color.png: 384x640 1 box, 884.4ms
Speed: 1.1ms preprocess, 884.4ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)


1


In [53]:
from crop_ply import *
xyz_output= cropPly(image_points,"validimage/5.ply")
print(xyz_output)

[PointCloud with 3588 points.]


In [104]:
import cv2
import pyrealsense2 as rs
import numpy as np
from ultralytics import YOLO

# RealSense 카메라 스트리밍 설정
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 360, rs.format.z16, 30)
pipeline.start(config)

# RealSense 정렬 프로세서 생성
align_to = rs.stream.color
align = rs.align(align_to)

# YOLO 모델 로드 (예: Darknet 또는 다른 라이브러리 사용)
model = YOLO("modelFolder/model_v7.pt")

# 클래스별 고유한 색상 정의
class_colors = {
    0: (0, 255, 0),   # 클래스 0의 색상 (예: 녹색)
    1: (255, 0, 0),   # 클래스 1의 색상 (예: 빨강)
    2: (0, 0, 255),   # 클래스 2의 색상 (예: 파랑)
    3: (255, 255, 0), # 클래스 3의 색상 (예: 노랑)
    4: (128, 0, 128), # 클래스 4의 색상 (예: 보라)
}

while True:
    # RealSense 카메라 프레임 가져오기
    frames = pipeline.wait_for_frames()
    
    # 컬러 이미지와 깊이 이미지 정렬
    aligned_frames = align.process(frames)
    aligned_color_frame = aligned_frames.get_color_frame()
    aligned_depth_frame = aligned_frames.get_depth_frame()

    if not aligned_color_frame or not aligned_depth_frame:
        continue

    # 정렬된 컬러 이미지와 깊이 이미지를 OpenCV 이미지로 변환
    color_image = np.asanyarray(aligned_color_frame.get_data())
    depth_image = np.asanyarray(aligned_depth_frame.get_data())

    # YOLO를 사용하여 객체 검출 및 세그멘테이션 수행
    results = model(color_image, conf=0.8, device="cpu")
    
    # 결과를 color_image와 depth_image에 오버레이
    for result in results:
        # print("mask",result.masks)
        for box, cls, conf in zip(result.boxes.xywh.tolist(), result.boxes.cls.tolist(), result.boxes.conf.tolist()):
            x, y, w, h = box
            x1, y1, x2, y2 = int(x - w / 2), int(y - h / 2), int(x + w / 2), int(y + h / 2)
            label = f"{model.names[int(cls)]}: {conf:.2f}"
            color_image = cv2.rectangle(color_image, (x1, y1), (x2, y2), class_colors[cls], 2)
            color_image = cv2.putText(color_image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            depth_image = cv2.rectangle(depth_image, (x1, y1), (x2, y2), class_colors[cls], 2)

    # 컬러 이미지를 따로 표시
    cv2.imshow("Color Image", color_image)

    # 깊이 이미지 시각화
    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.1), cv2.COLORMAP_JET)
    cv2.imshow("Depth Image", depth_colormap)

    # 'q' 키를 눌러 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 정리 및 종료
pipeline.stop()
cv2.destroyAllWindows()



0: 384x640 1 box, 922.4ms
Speed: 2.1ms preprocess, 922.4ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 box, 984.4ms
Speed: 0.0ms preprocess, 984.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 box, 947.3ms
Speed: 7.2ms preprocess, 947.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 box, 876.5ms
Speed: 0.0ms preprocess, 876.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 box, 986.5ms
Speed: 0.0ms preprocess, 986.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 box, 924.1ms
Speed: 0.0ms preprocess, 924.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 box, 930.7ms
Speed: 0.0ms preprocess, 930.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 box, 928.4ms
Speed: 2.0ms preprocess, 928.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


In [19]:
import open3d as o3d
import numpy as np


# 포인트 클라우드를 시각화
for cloud in xyz_output:
    # 포인트 클라우드의 경계 상자 계산
    aabb = cloud.get_axis_aligned_bounding_box()
    aabb.color = (1, 0, 0)

    obb = cloud.get_oriented_bounding_box()
    obb.color = (0, 1, 0)

    # 클라우드 포인트의 중심 위치 계산
    center = np.mean(np.asarray(cloud.points), axis=0)
    print(center)

    # 중앙 위치에 좌표 프레임 메시 생성
    mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.07)  # size를 조절하여 좌표 프레임 크기 설정 가능

    # 중앙 위치에 좌표 프레임 메시를 이동
    mesh.translate(center)

    # 시각화
    o3d.visualization.draw_geometries([cloud, aabb, obb, mesh])

    # PCA를 수행하여 주요 주성분 벡터를 찾습니다.
    points = np.asarray(cloud.points)
    covariance_matrix = np.cov(points, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)

    # 주요 주성분 벡터와 각각의 축과의 각도를 계산합니다.
    x_axis = np.array([1, 0, 0])  # x 축
    y_axis = np.array([0, 1, 0])  # y 축
    z_axis = np.array([0, 0, 1])  # z 축

    angle_x_rad = np.arccos(np.dot(eigenvectors[:, -1], x_axis) / (np.linalg.norm(eigenvectors[:, -1]) * np.linalg.norm(x_axis)))
    angle_x_deg = np.degrees(angle_x_rad)

    angle_y_rad = np.arccos(np.dot(eigenvectors[:, -1], y_axis) / (np.linalg.norm(eigenvectors[:, -1]) * np.linalg.norm(y_axis)))
    angle_y_deg = np.degrees(angle_y_rad)

    angle_z_rad = np.arccos(np.dot(eigenvectors[:, -1], z_axis) / (np.linalg.norm(eigenvectors[:, -1]) * np.linalg.norm(z_axis)))
    angle_z_deg = np.degrees(angle_z_rad)

    
    print(f"(x 축 기준): {angle_x_deg}도")
    print(f"(y 축 기준): {angle_y_deg}도")
    print(f"(z 축 기준): {angle_z_deg}도")
    
    o3d.io.write_point_cloud("robot/test.ply", cloud)


[  -0.039454   -0.023009    -0.64544]
(x 축 기준): 142.94969940274632도
(y 축 기준): 127.03310642746156도
(z 축 기준): 90.97338359234107도


In [103]:
import open3d as o3d
import numpy as np
from ultralytics import YOLO
from crop_ply import *

model = YOLO("modelFolder/model_v7.pt")
results = model(source="validimage/3_Color.png",device="cpu", project = "results",name="test6d", conf=0.8 )
for result in results:
    image_points = result.masks.xyn
    print(len(image_points))


xyz_output= cropPly(image_points,"validimage/3.ply")
print(xyz_output)


# 포인트 클라우드를 시각화
for cloud in xyz_output:
    # 포인트 클라우드의 경계 상자 계산
    aabb = cloud.get_axis_aligned_bounding_box()
    aabb.color = (1, 0, 0)
    
    obb = cloud.get_oriented_bounding_box()
    obb.color = (0, 1, 0)

    # aabb의 중심 위치 계산
    center = np.mean(np.asarray(cloud.points), axis=0)

    # # 중앙 위치에 좌표 프레임 메시 생성
    mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.01)  # size를 조절하여 좌표 프레임 크기 설정 가능
    mesh.translate(center)

    # aabb의 중점을 지나는 X, Y, Z 축 선을 시각화
    aabb_x_axis = o3d.geometry.LineSet()
    aabb_x_axis.points = o3d.utility.Vector3dVector([center, [center[0] + 0.1, center[1], center[2]]])
    aabb_x_axis.lines = o3d.utility.Vector2iVector([[0, 1]])
    aabb_x_axis.colors = o3d.utility.Vector3dVector([[0, 0, 0]])

    aabb_y_axis = o3d.geometry.LineSet()
    aabb_y_axis.points = o3d.utility.Vector3dVector([center, [center[0], center[1] + 0.1, center[2]]])
    aabb_y_axis.lines = o3d.utility.Vector2iVector([[0, 1]])
    aabb_y_axis.colors = o3d.utility.Vector3dVector([[0, 0, 0]])

    aabb_z_axis = o3d.geometry.LineSet()
    aabb_z_axis.points = o3d.utility.Vector3dVector([center, [center[0], center[1], center[2] + 0.1]])
    aabb_z_axis.lines = o3d.utility.Vector2iVector([[0, 1]])
    aabb_z_axis.colors = o3d.utility.Vector3dVector([[0, 0, 0]])

    # obb의 방향 벡터를 사용하여 X, Y, Z 축 선을 시각화
    obb_x_axis = o3d.geometry.LineSet()
    obb_x_axis.points = o3d.utility.Vector3dVector([center, center + 0.1 * obb.R[0, :3]])  # 첫 번째 방향 벡터는 X 축
    obb_x_axis.lines = o3d.utility.Vector2iVector([[0, 1]])
    obb_x_axis.colors = o3d.utility.Vector3dVector([[1, 0, 0]])

    obb_y_axis = o3d.geometry.LineSet()
    obb_y_axis.points = o3d.utility.Vector3dVector([center, center + 0.1 * obb.R[1, :3]])  # 두 번째 방향 벡터는 Y 축
    obb_y_axis.lines = o3d.utility.Vector2iVector([[0, 1]])
    obb_y_axis.colors = o3d.utility.Vector3dVector([[0, 1, 0]])

    obb_z_axis = o3d.geometry.LineSet()
    obb_z_axis.points = o3d.utility.Vector3dVector([center, center + 0.1 * obb.R[2, :3]])  # 세 번째 방향 벡터는 Z 축
    obb_z_axis.lines = o3d.utility.Vector2iVector([[0, 1]])
    obb_z_axis.colors = o3d.utility.Vector3dVector([[0, 0, 1]])

    # # OBB 박스의 방향 벡터 추출
    obb_rotation_matrix = obb.R
    obb_x_axis1 = obb_rotation_matrix[:, 0]  # X 축 방향 벡터
    obb_y_axis1 = obb_rotation_matrix[:, 1]  # Y 축 방향 벡터
    obb_z_axis1 = obb_rotation_matrix[:, 2]  # Z 축 방향 벡터

    # AABB 박스의 각 축 벡터
    aabb_x_axis1 = np.array([1.0, 0.0, 0.0])  # AABB X 축
    aabb_y_axis1 = np.array([0.0, 1.0, 0.0])  # AABB Y 축
    aabb_z_axis1 = np.array([0.0, 0.0, 1.0])  # AABB Z 축

    # 두 박스 간의 각도 계산
    angle_x = np.arccos(np.dot(obb_x_axis1, aabb_x_axis1))
    angle_y = np.arccos(np.dot(obb_y_axis1, aabb_y_axis1))
    angle_z = np.arccos(np.dot(obb_z_axis1, aabb_z_axis1))

    # 라디안을 도(degree)로 변환
    angle_x_deg = np.degrees(angle_x)
    angle_y_deg = np.degrees(angle_y)
    angle_z_deg = np.degrees(angle_z)

    print("Angle between OBB X axis and AABB X axis:", angle_x_deg)
    print("Angle between OBB Y axis and AABB Y axis:", angle_y_deg)
    print("Angle between OBB Z axis and AABB Z axis:", angle_z_deg)
    
    print("roll:",angle_z_deg)
    print("pitch:",angle_y_deg)
    print("yaw",angle_x_deg)

    # 시각화
    o3d.visualization.draw_geometries([cloud, aabb, obb,aabb_x_axis, aabb_y_axis, aabb_z_axis, obb_x_axis, obb_y_axis, obb_z_axis])



image 1/1 c:\Team_Project-1\validimage\3_Color.png: 384x640 1 box, 1018.5ms
Speed: 0.0ms preprocess, 1018.5ms inference, 15.1ms postprocess per image at shape (1, 3, 384, 640)


1
[PointCloud with 1508 points.]
Angle between OBB X axis and AABB X axis: 86.63452049817697
Angle between OBB Y axis and AABB Y axis: 86.62798397036205
Angle between OBB Z axis and AABB Z axis: 3.745617533598035
roll: 3.745617533598035
pitch: 86.62798397036205
yaw 86.63452049817697


In [20]:
import open3d as o3d
import numpy as np
from ultralytics import YOLO
from crop_ply import *

model = YOLO("modelFolder/model_v7.pt")
results = model(source="validimage/4_Color.png",device="cpu", project = "results",name="test6d", conf=0.8 )
for result in results:
    image_points = result.masks.xyn
    print(len(image_points))


xyz_output= cropPly(image_points,"validimage/4.ply")
print(xyz_output)

# 포인트 클라우드를 시각화
for cloud in xyz_output:
    # # 포인트 클라우드의 경계 상자 계산
    # aabb = cloud.get_axis_aligned_bounding_box()
    # aabb.color = (1, 0, 0)

    # obb = cloud.get_oriented_bounding_box()
    # obb.color = (0, 1, 0)

    # 클라우드 포인트의 중심 위치 계산
    center = np.mean(np.asarray(cloud.points), axis=0)
    print(center)
    
     # PCA를 수행하여 주요 주성분 벡터를 찾습니다.
    points = np.asarray(cloud.points)
    covariance_matrix = np.cov(points, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
   # 가장 큰 고유값에 해당하는 주요 주성분(회전 방향) 확인
    main_principal_component = eigenvectors[:, 2]  # 세 번째 고유벡터 선택
    # 주요 주성분의 방향 확인 (이 예제에서는 회전 정보를 출력합니다)
    rotation_angle_x = np.arctan2(main_principal_component[2], main_principal_component[1])
    rotation_angle_y = np.arctan2(main_principal_component[0], main_principal_component[2])
    rotation_angle_z = np.arctan2(main_principal_component[1], main_principal_component[0])
    # 각도를 라디안에서 도로 변환하여 출력
    rotation_angle_x_deg = np.degrees(rotation_angle_x)
    rotation_angle_y_deg = np.degrees(rotation_angle_y)
    rotation_angle_z_deg = np.degrees(rotation_angle_z)

    print("X 축 주위 회전 각도 (도):", rotation_angle_x_deg)
    print("Y 축 주위 회전 각도 (도):", rotation_angle_y_deg)
    print("Z 축 주위 회전 각도 (도):", rotation_angle_z_deg)

   

     # 주요 주성분 벡터와 중심 위치를 시각화
    lines = []
    for i in range(3):  # 주요 주성분 벡터의 개수는 3개입니다.
        eigenvector = eigenvectors[:, i]  # i번째 주요 주성분 벡터 가져오기

        # 중심 위치에서 주요 주성분 벡터를 시작하도록 끝점 좌표 계산
        endpoint = center + eigenvector * 0.1  # 벡터의 크기를 조절하여 시각화 크기 설정 가능

        # 주요 주성분 벡터를 라인 메시로 생성하여 시각화
        line = o3d.geometry.LineSet()
        line.points = o3d.utility.Vector3dVector(np.array([center, endpoint]))
        line.lines = o3d.utility.Vector2iVector(np.array([[0, 1]]))

        # 주요 주성분 벡터에 따라 다른 색상 설정
        if i == 0:
            line.colors = o3d.utility.Vector3dVector(np.array([[1, 0, 0]]))  # 빨간색
        elif i == 1:
            line.colors = o3d.utility.Vector3dVector(np.array([[0, 1, 0]]))  # 녹색
        elif i == 2:
            line.colors = o3d.utility.Vector3dVector(np.array([[0, 0, 1]]))  # 파란색
        
        lines.append(line)
    

      # 시각화에 추가
    # o3d.visualization.draw_geometries([cloud, aabb, obb, *lines])
    o3d.visualization.draw_geometries([cloud, *lines])

    # 포인트 클라우드를 PLY 파일로 저장
    o3d.io.write_point_cloud("robot/test.ply", cloud)


image 1/1 c:\Team_Project-1\validimage\4_Color.png: 384x640 1 box, 896.7ms
Speed: 0.0ms preprocess, 896.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


1
[PointCloud with 1516 points.]
[  -0.039454   -0.023009    -0.64544]
X 축 주위 회전 각도 (도): -178.3843318638702
Y 축 주위 회전 각도 (도): -91.21937282077778
Z 축 주위 회전 각도 (도): -142.96065447294583
